In [ ]:
from typing import List, Tuple
from datetime import datetime, timedelta
import os
import cv2
import pickle
import itertools
import json
import multiprocessing
from math import nan

import altair as alt
import numpy.typing as npt
import numpy as np
import pandas as pd
from tqdm import tqdm

from utils.compose_factories.compose import compose_factory
from utils.data_dirs import data_dirs
from utils.get_videos import get_videos
from utils.frame_json_encoder import FrameJSONEncoder
from utils.typings import Frame
from utils.load_images import load_images

In [ ]:
raw_dir, processed_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(False)
print(raw_dir)
print(processed_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

In [ ]:
fps = 12

In [ ]:
interpolate = False

In [ ]:
compose = compose_factory(fps, raw_dir, processed_dir, interpolate)

In [ ]:
sample_data = pd.read_pickle(os.path.join(processed_dir, 'sample_data.pkl'))

In [ ]:
videos = get_videos(sample_data)

In [ ]:
output = {}
misalignments = []
for i, name in enumerate(videos):
    print(f'{i} / {len(videos)}:', name)
    video = videos[name]
    
    filenames = []
    for f in video:
        filenames.append(f.filename)
    img_cache = load_images(raw_dir, filenames)

    assert len(set(f.location for f in video)) == 1, set(f.location for f in video)
    frames, filename, _misalignments = compose(video[0].location, name, video, img_cache)
    o = {
        "filename": filename,
        "start": video[0].timestamp,
        "location": video[0].location,
        "columns": [
            "cameraId",
            "frameId",  # sample data token
            "frameNum",
            "filename",
            "cameraTranslation",
            "cameraRotation",
            "cameraIntrinsic",
            "egoTranslation",
            "egoRotation",
            "timestamp",
            "cameraHeading",
            "egoHeading",
            "location",
        ],
        "frames": [
            tuple(video[f])
            for img, f in frames
        ]
    }
    output[name] = o
    print("done", filename)
base = os.path.join(processed_dir, 'videos')
with open(os.path.join(base, 'frames.pickle'), "wb") as f:
    pickle.dump(output, f)

with open(os.path.join(base, 'frames.pkl'), "wb") as f:
    pickle.dump(output, f)

with open(os.path.join(base, 'frames.json'), "w") as f:
    json.dump(output, f, indent=2, sort_keys=True, cls=FrameJSONEncoder)